## Scenario

You are a data analyst at a sports news website. You are responsible for gathering sports statistics. The company requires automated scripts to periodically update game data. You have gathering sports statistics from a sports API and automating the delivery of the data to your desktop every morning. Until now, you have been manually making note of each set of stats that gets delivered to your desktop. To make your recording process easier and less prone to errors, you have been asked to create a process that automatically records the sports data in an Excel spreadsheet each morning.


## Objective

The goal of this project is to have you apply maintenance best practices to an automation script, to improve efficiency and troubleshoot errors. You will also implement basic logging and make a small code modification to address a potential issue.

Please complete all steps within this notebook as you follow the instructions in the assignment.

In [ ]:
import requests
from plyer import notification
import schedule
import time
# Get your API key from https://www.football-data.org/ and replace 'YOUR_API_KEY_HERE'
api_key = 'YOUR_API_KEY_HERE'
headers = {'X-Auth-Token': api_key}
url = "http://api.football-data.org/v4/matches"

def fetch_scores():
    response = requests.get(url, headers=headers)
    try:
        data = response.json()
        # Check if the 'matches' key exists in the response
        if 'matches' in data and len(data['matches']) > 0:
            match = data['matches'][0]
            score_line = f"{match['score']['fullTime']['homeTeam']} - {match['score']['fullTime']['awayTeam']}"
            team_home = match['homeTeam']['name']
            team_away = match['awayTeam']['name']
            return f"{team_home} {score_line} {team_away}"
        else:
            return "No matches currently available."
    except Exception as e:
        return f"Error fetching data: {e}"
        
fetch_scores()

In [ ]:
import openpyxl
import datetime


def send_notification():
    match_score = fetch_scores()
    notification.notify(
        title='Live Sports Score Update',
        message=match_score,
        app_icon=None,  # Path to an app icon
        timeout=10,  # Duration in seconds
    )
    # Call record_to_excel here, after sending the notification
    record_to_excel(match_score) 

def record_to_excel(match_data):
    """Records the match data to an Excel spreadsheet."""
    try:
        workbook = openpyxl.load_workbook('sports_data.xlsx')
        sheet = workbook.active
    except FileNotFoundError:
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet.append(["Home Team", "Score", "Away Team", "Date"])

    # Extract data (assuming match_data is in the format "HomeTeam Score - Score AwayTeam")
    try:
        home_team, score, away_team = match_data.split(' ')
        today = datetime.date.today().strftime("%Y-%m-%d")
        # Corrected line: Appending the complete date string
        sheet.append([home_team, score, away_team, today])  
    except ValueError:
        sheet.append(["Error processing data", "", "", today])

    workbook.save('sports_data.xlsx')

# Test sending a notification
send_notification()

schedule.every().day.at("08:00").do(send_notification)

# Test the scheduling of notifications (this will run until you stop it)
while True:
    schedule.run_pending()
    time.sleep(1)